In [2]:
print()

In [ ]:
# %%time
# LOAD PACKAGES
# -------------
# import Pkg; Pkg.add("CSV"); Pkg.add("GLPK"); Pkg.add("JuMP")
using JuMP  # Need to say it whenever we use JuMP
using GLPK  # Loading the GLPK module for using its solver
using CSV

# MODEL CONSTRUCTION
# ------------------
capstone = Model(with_optimizer(GLPK.Optimizer, tm_lim=10*1000)) 
# We also set the time limit, in milliseconds

In [ ]:
# INPUT DATA
# ----------

# Data on student preferences
unit_costs = CSV.read("../preprocessing/unit_costs.csv",header=false)
print(size(unit_costs))

# Data on Pillars
capacities = CSV.read("../preprocessing/capacities.csv",header=false)
print(size(capacities))

# Data on Pillars
supplies = CSV.read("../preprocessing/supplies.csv",header=false)
print(size(supplies))

length = size(unit_costs)[1]-1
iterlen = 1:length

In [ ]:
# VARIABLES
# ---------

# binary variable taking value 1 if there is a flow
@variable(capstone, flowing[i=iterlen,j=iterlen] >= 0, Bin)

# the flow
@variable(capstone, flow[i=iterlen,j=iterlen] >= 0, Int)  

print()

In [ ]:
# CONSTRAINTS
# -----------

# flow can be nonzero only if flowing is 1
@constraint(capstone, smaller_1[i=iterlen, j=iterlen], 
    flow[i,j] <= flowing[i,j]*9999999)

# flow can be nonzero only if capacity is nonzero
@constraint(capstone, smaller_2[i=iterlen, j=iterlen], 
    flow[i,j] <= capacities[i,j]*9999999)

# # Every node can only have one output
@constraint(capstone, output[i=iterlen], 
    sum(flowing[i,j] for j = iterlen) == 1)

# Every node needs to exhaust its supply
@constraint(capstone, supplied[i=iterlen], 
    sum(flow[i,j] for j = iterlen) - sum(flow[j,i] for j = iterlen) == supplies[i,1])

print()

In [ ]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Min, 
    sum(unit_costs[i,j]*flowing[i,j] for i = iterlen, j = iterlen))

print()

In [ ]:
# SOLVE IT
# --------

status = optimize!(capstone)

In [ ]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(capstone)) 